# Advent of Code 2021 - Day 3
> My solution and thoughts

- toc: false
- badges: false
- comments: false
- categories: [advent-of-code]

# Day 3
On the third day of [AoC](https://adventofcode.com/2021/day/3) my data gave to me... a long list of binary.

## Part 1
This problem is to use a list of binary to generate two binary numbers, then multiply them. That means that this could be done without converting the list to actual binary, though there's probably a clever way to get the answer if we do.

The first number is composed of the most common bit from each position. So if we have:

```
1011
0010
0110
```

Then we would build the number `0010`, since those are the most common bits in each column position. The second number is made from the _least_ common bit, which just means that we flip the bits of the first and boom, we have it.

With that said, let's build a simple solution:

In [195]:
with open('../assets/inputs/aoc/2021/day_3.txt', 'r') as data_file:
    lines = [line.strip() for line in data_file.readlines()]

display(lines[:3], lines[997:])

['001001100101', '010100011100', '100000110001']

['011000110110', '101110011011', '110101100001']

**side note:** I don't think that we're going to get the opportunity to do so here, but you might be interested in trying to figure out how you could re-write any of these answers such that we aren't required to read the whole data set into memory. It's a fun challenge!

So now we have our lines, and the naive approach is just sitting there waiting for us! What we would need to do if we did this by hand is mark down a count of each bit in each position. I migth use tally marks, but since we're pythonistas we could use a collections counter. The only issue is that we want to use it on the columns.

Because the data we want is column-wise and we left the values as strings, we _could_ just iterate the list a few times to get it. I'm lazy, so I'm going to do that first.

In [196]:
from collections import Counter

elem_len = len(lines[0])
list_len = len(lines)

columns = [[lines[i][j] for i in range(list_len)] for j in range(elem_len)]
counts = [Counter(column) for column in columns]

display(counts)

[Counter({'0': 528, '1': 472}),
 Counter({'0': 479, '1': 521}),
 Counter({'1': 485, '0': 515}),
 Counter({'0': 499, '1': 501}),
 Counter({'0': 495, '1': 505}),
 Counter({'1': 515, '0': 485}),
 Counter({'1': 512, '0': 488}),
 Counter({'0': 502, '1': 498}),
 Counter({'0': 507, '1': 493}),
 Counter({'1': 507, '0': 493}),
 Counter({'0': 504, '1': 496}),
 Counter({'1': 481, '0': 519})]

Ok, we have the positional counts of each thing like we might do by hand. We just need to transform this into an actual value, or two. These will need to be multiplied, so we better make them actual binary values. Then we'll need to positionally set their bits. We could also just compute the binary value as a string and use the `int` function, but this method does a little bitwise operator fun that I think is interesting to try out once in a while:

In [197]:
powers = [2**n for n in range(elem_len - 1, -1, -1)]
gamma = 0
epsilon = 0

for i, count in enumerate(counts):
    if count['1'] > count['0']:
        gamma |= powers[i]
    else:
        epsilon |= powers[i]

display(gamma * epsilon)

3901196

We spin up a list of the powers of two, or each position's bit value of 1. We want to step through these in reverse order, since the puzzle considers the first bit to be the largest value. Then we use a bitwise-or on the current value, and since we start out with 0 and are moving in descending order, we know we're going to flip a 0 to a 1 for whichever number needs to be altered.

Part 1 is done! This is probably the least efficient way that I could do this, but we have an answer.

## Part 2
Part two introduces a crazy new approach. We need to parse the list down to a single number, but instead of looking at the most common bit to keep that bit, we're going to look at the most/least common bit and keep all the numbers. This process will then be applied to this new, shorter list. There are two values that we want to produce this way: `og_rating` and `cs_rating`. I picked these variables based on the fact that they're called (O)xygen (G)enerator rating and (C)O2 (S)crubber rating in the puzzle. The rules for generation are as follows:

The `og_rating` asks us to keep the values with a 1 in the position we're evaluating if 0 and 1 appear evenly.
The `cs_rating` asks us to keep the values with a 0 in the position we're evaluating if 0 and 1 appear evenly.
The `og_rating` will keep numbers whose value in the current position is the **most common bit**
The `cs_rating` will keep numbers whose value in the current position is the **least common bit**

**side note:** These rules ensure that we will always shrink the list. However, it does strike me that you might be able to compose a list that doesn't get reduced to a single value in the course of applying this rule. I wonder what that might look like? What are the limits on the size of the list vs the size of it's elements such that you might be able to guarantee a value is generated?

The thing that we notice here is that the process we used before lost a lot of information. That is, my `counts` variable is useless for this part of the challenge. I know the most common bit, but I lost the relationship to the individual numbers in the list. Thankfully, the input here is small, so I can just iterate the numbers to create a new list based on the knowledge of what the most common things are. But then I'll need to create _new_ positional counts. So is that really the best way?

The common problem here is to find the most common bit for a given position and a list of binary values. So we should try and do that as efficiently as possible:

In [198]:
def most_common_for_position(bin_values, position):
    zero_count = 0
    ones_count = 0
    for bn in bin_values:
        if bn[position] == '0':
            zero_count += 1
        else:
            ones_count += 1

    if zero_count > ones_count:
        return '0'
    elif ones_count > zero_count:
        return '1'
    elif ones_count == zero_count:
        return 'eq'

The tricky part here is that we're using 0-indexing. I find that switching to that thinking early in the process is helpful because python itself is 0-indexed. If we were using Julia, or another 1-indexed language, then we could rely on the concept. That said, problems that are one indexed in their language are always something to look out for. You know what they say, there are 2 hard problems in computer science: cache invalidation, naming things, and off-by-one errors...

At any rate, I went ahead and added an explicit case for the equality of the numbers. This will come in handy for that tie-breaker rule set. At this point it might be nice to validate our function by soliving part one again but in a different way:

In [199]:
def invert(str_bin):
    return ''.join(['0' if char == '1' else '1' for char in str_bin])

def to_num(n):
    return int(n, 2)
        
gamma = ''
for i in range(elem_len):
    gamma += most_common_for_position(lines, i)

epsilon = invert(gamma)
display(to_num(gamma) * to_num(epsilon))

3901196

I would have used a binary operator for the inversion, but with signed binary numbers it was being weird. I'm not used to dealing with binary, so I just opted to write a simple helper to deal with it. But we see that the function works!

At this point we have a few steps to follow:
1. Get the most common value for the position
2. Filter the list into a new list

Then we do it again, but with the position incremented. There are two ways to implement this type of logic: iterarive, and recursive. I tend to prefer iteration over recursion in python due to the stack size. I also find that things defined recursively are used easily as iterators, which are easiest to implement as iterative. So I'd even implement fibonacci numbers this way. So let's get the `og_rating` sorted:

In [200]:
position = 0
binary_list = lines.copy()

while len(binary_list) > 1:
    most_common = most_common_for_position(binary_list, position)

    if most_common == 'eq':
        keep = '1'
    else:
        keep = most_common

    binary_list = [line for line in binary_list if line[position] == keep]
    position += 1

og_rating = binary_list[0]

display(og_rating)

'011001100111'

We use essentially the same logic for the `cs_rating` but we insert a bit flip for the concept of "least common" in the `cs_rating` specification. Of course, doing this with actual bits would be a "bit" better, but we have strings, so I'mma use strings.

In [201]:
position = 0
binary_list = lines.copy()

while len(binary_list) > 1:
    most_common = most_common_for_position(binary_list, position)

    if most_common == 'eq':
        keep = '0'
    else:
        keep = invert(most_common)

    binary_list = [line for line in binary_list if line[position] == keep]
    position += 1

cs_rating = binary_list[0]

In [202]:
display(to_num(og_rating) * to_num(cs_rating))

4412188

Boom! There we go!

Now, this is obviously really ugly, and cleaning it up is an interesting prospect. The big issue is, we have a new list and new state related to it on every iteration. So we have to keep learning what the new most common or least common entry is for each position. If it weren't for that, we could filter the list with some globally accessible state. But alas...

What we could do is wrap the whole thing in a function, but honestly that feels ugly too. Just look at this:

In [203]:
def get_rating(binary_input, rating_type='og_rating'):
    position = 0
    binary_list = binary_input.copy()

    while len(binary_list) > 1:
        most_common = most_common_for_position(binary_list, position)
        
        if rating_type == 'og_rating':
            if most_common == 'eq':
                keep = '1'
            else:
                keep = most_common
        else:
            if most_common == 'eq':
                keep = '0'
            elif most_common == '1':
                keep = '0'
            else:
                keep = '1'

        binary_list = [line for line in binary_list if line[position] == keep]
        position += 1
        
    return binary_list[0]

display(to_num(get_rating(lines, rating_type='cs_rating')) * to_num(get_rating(lines)))

4412188

This is ugly. But so is what we're doing. Another option is to write the function, but give it the behavior we want by passing in a keep function. Then we remove the need for the `rating_type` param.

In [204]:
def og_rating_keep(most_common):
    return '1' if most_common == 'eq' else most_common
    
def cs_rating_keep(most_common):
    return '0' if most_common == 'eq' else invert(most_common)

def get_rating_hof(binary_input, keep_function):
    position = 0
    binary_list = binary_input.copy()

    while len(binary_list) > 1:
        most_common = most_common_for_position(binary_list, position)

        binary_list = [line for line in binary_list if line[position] == keep_function(most_common)]
        position += 1
        
    return binary_list[0]

display(to_num(get_rating_hof(lines, cs_rating_keep)) * to_num(get_rating_hof(lines, og_rating_keep)))

4412188

If you're not familiar with [higher order functions](https://en.wikipedia.org/wiki/Higher-order_function), then this will be new to you. Otherwise, this is an excellent way to clean up logic that you want to know at call time. The hallmark for this is passing in a logical flag. Anytime that you do that, you could be using a higher order function. Some people might term this a "callback", but that's the function that is _given_ to a higher order function. 

At any rate, this is probably the furthest I would go in trying to clean this up. We _could_ go a step further though...

We could try and generalize the filtering. What that means is that we have some rule that we want to apply to each successive iteration of the list itself. That might look something like this:

``` python
def apply_iterative_rule(some_list, rule):
    list_copy = some_list.copy()
    iteration = 0
    while len(list_copy) > 1:
        list_copy = rule(list_copy, iteration)
    return list_copy[0]
```

What we're defining here is the interface for the `rule` function. We know that to apply our current rule we need the current list state (`list_copy`) and the `position`. But position is the same as iteration, and iteration is more general.

The next challenge is to come up with the `rule` function itself. It should basically look like the inside of our loop. After all, that's where it's being called essentially.

``` python
def rating_rule(binary_list, position):
    most_common = most_common_for_position(binary_list, position)

    return [line for line in binary_list if line[position] == keep_function(most_common)]
```

This is close, but we don't have the keep function, and in the context of this function, we don't control the call site. That's internal to `apply_iterative_rule`. If we were to change that to allow us to pass in a keep function, we'd kind of break the generality. A little bit of [curry](https://en.wikipedia.org/wiki/Currying) could help the situation.

``` python
from functools import partial

def rule_base(keep_function, binary_list, position):
    most_common = most_common_for_position(binary_list, position)

    return [line for line in binary_list if line[position] == keep_function(most_common)]

og_rating_rule = partial(rule_base, og_rating_keep)
cs_rating_rule = partial(rule_base, cs_rating_keep)
```

Using [partial](https://docs.python.org/3/library/functools.html#functools.partial) allows us to apply a function argument, but instead of erroring out if there are missing arguments, it just returns a callable object that will apply arguments to the original function definition. Let's see how this works:

In [205]:
from functools import partial

def rule_base(keep_function, binary_list, position):
    most_common = most_common_for_position(binary_list, position)

    return [line for line in binary_list if line[position] == keep_function(most_common)]

og_rating_rule = partial(rule_base, og_rating_keep)
cs_rating_rule = partial(rule_base, cs_rating_keep)

def apply_iterative_rule(some_list, rule):
    list_copy = some_list.copy()
    iteration = 0
    while len(list_copy) > 1:
        list_copy = rule(list_copy, iteration)
        iteration += 1
    return list_copy[0]

display(to_num(apply_iterative_rule(lines, og_rating_rule)) * to_num(apply_iterative_rule(lines, cs_rating_rule)))

4412188

Ok, so this _looks_ really nice, but what immediately jumps out to me is that `apply_iterative_rule` depends on the rule actually reducing the list to a given value. That is, we can pass a perfectly operational rule to the function, and it will simply never terminate.

How could we avoid this? The simplest way I can think is to pass in a limit to the function. Something like:

``` python
def apply_iterative_rule(ls, rule, limit):
    ls_copy = ls.copy()
    iteration = 0
    while len(ls_copy) > limit:
        ls_copy = rule(list_copy, iteration)
        iteration += 1
    return ls_copy[0]
```

This allows us to pass in the rule, and some expectation about it. But is there actually any guarantee that we'll terminate? It depends on the rule. Since the rule itself is decoupled from this, it feels risky to me. Essentially, we need a way to guarantee that all the rules that we pass in will decrease...

I don't really know how one might do that. This is why I prefer stopping at the prior step. We generalized just enough to clean up the code and isolate the "business logic" of which thing we wanted to keep. We also kept the filtration logic visible within the loop that controls it. That means that we can, just by looking at the `get_rating_hof` function, be relatively sure that we'll terminate.


# Conclusion
This was a fun one. It gave me a chance to use partials again, and I'm a huge fan of higher order functions. It feels like there may be a more general solution, but we will always sacrifice efficiency. On that note, it also seems like there is probably a pretty clever way to solve this in a super efficient way using some [evil bit-level hacking](https://en.wikipedia.org/wiki/Fast_inverse_square_root). I don't generally go in for that. The code solves the problem, and it does so quickly enough for me to write it up.

I will, however, come back to the topic of termination. I feel like looking at the rules we're applying vs the input we take in is interesting enough to warrant that. As it is, our solution seems to depend on the fact that we're playing a game that must have a solution. In the real world, being able to tell if the data won't terminate is pretty valuable. I'll try to remember to update with a link to that when the time comes.